In [209]:
import pandas as pd
import numpy as np
import pyarrow.dataset as ds
import s3fs

## Dataset Import

In [237]:
# Import FIP Dataset

s3_path_fip = (
    "s3://m3-intel-hub-dp-us-east-1-517292-prod/"
    "publish/data-product/financial_inventory_projection_report_network_update/"
)

dataset = ds.dataset(
    s3_path_fip,
    format="parquet",
    partitioning="hive" 
)

table = dataset.to_table(
    filter=(
        (ds.field("corporate_brand") == "REVLIMID") &
        (ds.field("snapshot_date") > "2025-07-01") &       
        (ds.field("date") == "202612") &                    # filter only 2026 YE FIP data
        ~(
            (ds.field("snapshot_date") == "2026-01-23") &
            (ds.field("snapshot_type") == "friday")         # drop 23rd Fri snapshot data to avoid double counting
        )
    )
)

df_fip = table.to_pandas()
df_fip.head()

,material,plant,date,quantity,total_cost,concost_source,unit_of_measure,cost_per_unit,source,corporate_brand,material_type,development_lifecycle_status,enterprise_category,enterprise_sub_category,dosage_form_parent,corp_brand_id,network_or_business_unit,snapshot_type,snapshot_date
0,1458497,2061,202612,865.2,2859.01014,concost dp,ST,3.30445,rr,REVLIMID,PACK,COMMERCIAL,PACKAGE COMPONENT,PACKAGE COMPONENT,nan,03302101,PHARMA,friday,2025-07-18
1,1424858,2023,202612,0.0,0.00000,concost dp,ST,24.20032,rr,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,friday,2025-07-18
2,1428613,2021,202612,0.0,0.00000,concost dp,ST,50.58223,rr,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,friday,2025-07-18
3,1431792,2021,202612,59.0,6391.59036,concost dp,ST,108.33204,rr,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,friday,2025-07-18
4,1431878,2023,202612,0.0,0.00000,concost dp,ST,117.51409,rr,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,friday,2025-07-18


In [233]:
df_fip.shape

(34823, 19)

In [214]:
# Import Plant Type Data

s3_path_plants = (
    "s3://m3-intel-hub-dp-us-east-1-517292-prod/"
    "refined/data-asset/fin_inv_proj/"
    "bms_internal_vs_external_plants/"
    "bms_internal_vs_external_plants.parquet"
)

df_plants = pd.read_parquet(s3_path_plants)
#df_plants.head()

In [215]:
# Import node type Dataset

fs = s3fs.S3FileSystem()  # uses SageMaker execution role

parquet_files_nt = fs.glob(
    "s3://m3-intel-hub-dp-us-east-1-517292-prod/"
    "dbt_intelligence_hub/intelligence_hub_db_sandbox_staging/"
    "src__sap_t001w/data/*.parquet"
)

df_ntype = pd.read_parquet(
    parquet_files_nt,
    engine="pyarrow",
    dtype_backend="pyarrow",
    filesystem=fs
)

#df_ntype.head()

In [216]:
# Import Material Master Data

fs = s3fs.S3FileSystem()  # uses SageMaker execution role

parquet_files = fs.glob(
    "s3://m3-intel-hub-dp-us-east-1-517292-prod/"
    "refined/data-asset/fin_inv_proj/"
    "sap_material_master/data/*.parquet"
)

df_mm = pd.read_parquet(
    parquet_files,
    engine="pyarrow",
    dtype_backend="pyarrow",
    filesystem=fs
)

#df_mm.head()

In [201]:
# import boto3

# s3 = boto3.client("s3")

# bucket = "m3-intel-hub-dp-us-east-1-517292-prod"
# prefix = "dbt_intelligence_hub/intelligence_hub_db_sandbox_staging/src__sap_t001w"

# response = s3.list_objects_v2(
#     Bucket=bucket,
#     Prefix=prefix
# )

# if "Contents" in response:
#     for obj in response["Contents"]:
#         print(obj["Key"], obj["Size"])
# else:
#     print("No objects found or no access.")

## Data Prep

In [238]:
# Create has_non_zero flag at material–plant level

df_fip["has_non_zero"] = (
    df_fip
    .groupby(["material", "plant"])["total_cost"]
    .transform(lambda x: (x != 0).any())
    .astype(int)
)

# Apply the filter
df_fip = df_fip.loc[df_fip["has_non_zero"] == 1].drop(columns="has_non_zero")

In [241]:
df_fip.shape

(28169, 19)

In [280]:
base1 = df_fip.copy()

In [281]:
# Join material master
mm_cols = [
    "material_number",
    "plant",
    "profit_center",
    #"corporate_brand",
    "brand_name",
    #"corp_brand_id",
    "material_description",
    #"material_type",
    "material_group",
    "old_material_number",
    "base_unit_of_measure",
    "unit_of_weight",
    #"development_lifecycle_status",
    "plant_specific_material_status",
    "mrp_type",
    "procurement_type",
    "safety_stock",
    "minimum_lot_size",
    "maximum_lot_size",
    "fixed_lot_size",
    "total_replenishment_lead_time",
    "total_shelf_life",
    "batch_management",
    "abc_indicator",
    #"valuation_class",
    #"price_unit",
    #"price_control_indicator",
]

material_master_sel = (
    df_mm[mm_cols]
    .drop_duplicates(subset=["material_number", "plant"])
)


base1 = base1.merge(
    material_master_sel,
    left_on=["material", "plant"],
    right_on=["material_number", "plant"],
    how="left",
    validate="m:1" 
)
base1 = base1.drop(columns=["material_number"])

base1 = base1.merge(
    df_plants[["Plant", "Plant Type"]],
    left_on=["plant"],
    right_on=["Plant"],
    how="left"
).drop(columns=["Plant"])

node_type_lkp = (
    df_ntype[["werks", "nodetype"]]
    .drop_duplicates(subset=["werks"])
)

base1 = base1.merge(
    node_type_lkp,
    left_on="plant",
    right_on="werks",
    how="left",
    validate="m:1"
).drop(columns=["werks"])

base1.shape

(28169, 39)

In [282]:
# rearrange columns for better readability
new_cols = [
    'corporate_brand',
    'material_type', 'development_lifecycle_status', 'enterprise_category',
    'enterprise_sub_category', 'dosage_form_parent', 'corp_brand_id',
    'network_or_business_unit',
    'profit_center', 'brand_name', 'material_description', 'material_group',
    'old_material_number', 'base_unit_of_measure', 'unit_of_weight',
    'plant_specific_material_status', 'mrp_type', 'procurement_type',
    'safety_stock', 'minimum_lot_size', 'maximum_lot_size',
    'fixed_lot_size', 'total_replenishment_lead_time', 'total_shelf_life',
    'batch_management', 'abc_indicator',
    'source', 'concost_source', 'Plant Type', 'nodetype',
    'unit_of_measure',
    'material', 'plant', 'date', 'quantity', 'total_cost',
    'cost_per_unit', 'snapshot_type', 'snapshot_date'
]

base1 = base1[new_cols]


In [283]:
# Add material entry flag 

base = base1.copy()
first_seen = (
    base.groupby(["material", "plant"])["snapshot_date"]
      .transform("min")
)

base["sku_status"] = np.where(
    base["snapshot_date"] == first_seen,
    "NEW",
    "EXISTING"
)


In [284]:
# fip copy df for data prep
df = base.copy()
df["snapshot_date"] = pd.to_datetime(df["snapshot_date"])


# snapshot lookup table
snapshot_calendar = (
    df[["snapshot_type", "snapshot_date"]]
    .drop_duplicates()
    .sort_values(["snapshot_type", "snapshot_date"])
)


# attach prev snapshot to snapshot calendar
snapshot_calendar["prev_snapshot_date"] = (
    snapshot_calendar
    .groupby("snapshot_type")["snapshot_date"]
    .shift(1)
)
snapshot_calendar      # comparing bd13 - bd13 snapshots and friday-friday snapshots. no bd13-fri snapshots

,snapshot_type,snapshot_date,prev_snapshot_date
5190,bd13,2025-08-26,NaT
8653,bd13,2025-09-18,2025-08-26
12884,bd13,2025-10-17,2025-09-18
17189,bd13,2025-11-19,2025-10-17
21462,bd13,2025-12-17,2025-11-19
26505,bd13,2026-01-23,2025-12-17
0,friday,2025-07-18,NaT
860,friday,2025-07-25,2025-07-18
1726,friday,2025-08-01,2025-07-25
2592,friday,2025-08-08,2025-08-01


In [285]:
# Attach previous snapshot date to each row

df = df.merge(
    snapshot_calendar[["snapshot_type", "snapshot_date", "prev_snapshot_date"]],
    on=["snapshot_type", "snapshot_date"],
    how="left"
)

In [286]:
df.head()

,corporate_brand,material_type,development_lifecycle_status,enterprise_category,enterprise_sub_category,dosage_form_parent,corp_brand_id,network_or_business_unit,profit_center,brand_name,...,material,plant,date,quantity,total_cost,cost_per_unit,snapshot_type,snapshot_date,sku_status,prev_snapshot_date
0,REVLIMID,PACK,COMMERCIAL,PACKAGE COMPONENT,PACKAGE COMPONENT,nan,03302101,PHARMA,10956,REVLIMID,...,1458497,2061,202612,865.2,2859.01014,3.30445,friday,2025-07-18,NEW,NaT
1,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,1428613,2021,202612,0.0,0.00000,50.58223,friday,2025-07-18,NEW,NaT
2,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,1431792,2021,202612,59.0,6391.59036,108.33204,friday,2025-07-18,NEW,NaT
3,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10001,REVLIMID,...,1428615,2053,202612,86.0,9672.68746,112.47311,friday,2025-07-18,NEW,NaT
4,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10001,REVLIMID,...,1431577,2058,202612,17.0,1862.65005,109.56765,friday,2025-07-18,NEW,NaT


In [287]:
# Prepare current and previous frames

# Current snapshot frame
current_df = df.copy()

current_df = current_df.rename(columns={
    "quantity": "quantity_curr",
    "cost_per_unit": "cost_per_unit_curr",
    "total_cost": "total_cost_curr",
})

# Previous snapshot frame
previous_df = df.rename(columns={
    "snapshot_date": "snapshot_date_prev",
    "quantity": "quantity_prev",
    "cost_per_unit": "cost_per_unit_prev",
    "total_cost": "total_cost_prev",
})[
    [
        "material",
        "plant",
        "date",
        "snapshot_type",
        "snapshot_date_prev",
        "quantity_prev",
        "cost_per_unit_prev",
        "total_cost_prev",
    ]
]


# Join current to previous snapshot
rca_base = current_df.merge(
    previous_df,
    left_on=[
        "material",
        "plant",
        "date",
        "snapshot_type",
        "prev_snapshot_date",
    ],
    right_on=[
        "material",
        "plant",
        "date",
        "snapshot_type",
        "snapshot_date_prev",
    ],
    how="left"
)

In [288]:
df.shape

(28169, 41)

In [289]:
# Flags for material-plant presence / absence between snapshots

# Not present in previous snapshot 
rca_base["is_new_in_current_snapshot"] = (
    rca_base["prev_snapshot_date"].notna() &
    rca_base["quantity_prev"].isna()
)

# Present in previous but missing in current snapshot
rca_base["is_new_in_current_snapshot"] = False


# Identify valid previous snapshots (from calendar logic)
valid_prev_snapshots = (
    snapshot_calendar["prev_snapshot_date"]
        .dropna()
        .unique()
)

# Restrict previous_df BEFORE the anti-join
previous_df_valid = previous_df[
    previous_df["snapshot_date_prev"].isin(valid_prev_snapshots)
]

# Present in previous but missing in current snapshot
# Identify rows present in previous snapshot but missing in current
prev_only = previous_df_valid.merge(
    current_df[
        [
            "material",
            "plant",
            "date",
            "snapshot_type",
            "prev_snapshot_date",
        ]
    ],
    left_on=[
        "material",
        "plant",
        "date",
        "snapshot_type",
        "snapshot_date_prev",
    ],
    right_on=[
        "material",
        "plant",
        "date",
        "snapshot_type",
        "prev_snapshot_date",
    ],
    how="left",
    indicator=True
).query("_merge == 'left_only'")


prev_only["is_dropped_in_current_snapshot"] = True

# Add missing columns for consistency
for col in rca_base.columns:
    if col not in prev_only.columns:
        prev_only[col] = np.nan

rca_base["is_dropped_in_current_snapshot"] = False

final_rca_frame = pd.concat(
    [rca_base, prev_only[rca_base.columns]],
    ignore_index=True
)

/tmp/ipykernel_11579/4268523466.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_rca_frame = pd.concat(


In [290]:
for col in [
    "is_new_in_current_snapshot",
    "is_dropped_in_current_snapshot",
]:
    final_rca_frame[col] = (
        final_rca_frame[col]
            .replace({1: True, 0: False})   # normalize numeric bools
            .fillna(False)                  # handle NaNs
            .astype("boolean")              # now safe
    )


/tmp/ipykernel_11579/1465783820.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)                  # handle NaNs


In [291]:
final_rca_frame.shape

(28371, 47)

In [292]:
final_rca_frame.tail()

,corporate_brand,material_type,development_lifecycle_status,enterprise_category,enterprise_sub_category,dosage_form_parent,corp_brand_id,network_or_business_unit,profit_center,brand_name,...,snapshot_type,snapshot_date,sku_status,prev_snapshot_date,snapshot_date_prev,quantity_prev,cost_per_unit_prev,total_cost_prev,is_new_in_current_snapshot,is_dropped_in_current_snapshot
28366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,...,friday,NaT,NaN,NaT,2026-01-09,458.000000,0.00001,0.004580,False,True
28367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,...,friday,NaT,NaN,NaT,2026-01-16,23.320000,163.98355,3824.096386,False,True
28368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,...,friday,NaT,NaN,NaT,2026-01-16,4.228232,106.24204,449.215993,False,True
28369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,...,friday,NaT,NaN,NaT,2026-01-16,55.480000,147.80652,8200.305730,False,True
28370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,...,friday,NaT,NaN,NaT,2026-01-16,0.000000,167.71521,0.000000,False,True


In [293]:
snapshot_mapping_check = (
    final_rca_frame
    .loc[:, ["snapshot_type", "snapshot_date", "prev_snapshot_date"]]
    .drop_duplicates()
    .sort_values(["snapshot_type", "snapshot_date"])
)

snapshot_mapping_check

,snapshot_type,snapshot_date,prev_snapshot_date
5190,bd13,2025-08-26,NaT
8653,bd13,2025-09-18,2025-08-26
12884,bd13,2025-10-17,2025-09-18
17189,bd13,2025-11-19,2025-10-17
21462,bd13,2025-12-17,2025-11-19
26505,bd13,2026-01-23,2025-12-17
28184,bd13,NaT,NaT
0,friday,2025-07-18,NaT
860,friday,2025-07-25,2025-07-18
1726,friday,2025-08-01,2025-07-25


Error handling

1. Division by zero & invalid math -
    Previous quantity = 0
    Previous cost = 0
    Previous FIP = 0
    Volatility = 0
2. Min rolling window
   less than min periods of 3
   Newly introduced SKU
   Flat history causing volatility = 0
3. double counting due to duplicates
4. missing prev snapshot data - nulls
   SKU appears for first time
   SKU disappears and reappears
   Previous quantity / cost not available
5. Extreme values in the history
    Very large quantities or costs in the past pushing the present numbers. The outliers in the past affects the current numbers. Exlcude the historical outliers
   

# Calculations

In [294]:
# Step 1: Compute raw change metrics (always runs)

# Base deltas

final_rca_frame["delta_quantity"] = (
    final_rca_frame["quantity_curr"] - final_rca_frame["quantity_prev"]
)

final_rca_frame["delta_cost_per_unit"] = (
    final_rca_frame["cost_per_unit_curr"] - final_rca_frame["cost_per_unit_prev"]
)


# Impact decomposition

# Quantity Impact 
final_rca_frame["quantity_impact"] = (
    final_rca_frame["delta_quantity"] *
    final_rca_frame["cost_per_unit_prev"]
)

# Cost Impact
final_rca_frame["cost_impact"] = (
    final_rca_frame["delta_cost_per_unit"] * final_rca_frame["quantity_prev"]
)

# Intercation
final_rca_frame["interaction_impact"] = (
    final_rca_frame["delta_quantity"] *
    final_rca_frame["delta_cost_per_unit"]
)

# Total Change
final_rca_frame["total_fip_change"] = (
    final_rca_frame["quantity_impact"] +
    final_rca_frame["cost_impact"] +
    final_rca_frame["interaction_impact"]
)

# Core Metrics

# delta_quantity_pct 
final_rca_frame["delta_quantity_pct"] = (
    final_rca_frame["delta_quantity"] /
    final_rca_frame["quantity_prev"]
)

# delta_cost_per_unit_pct
final_rca_frame["delta_cost_per_unit_pct"] = (
    final_rca_frame["delta_cost_per_unit"] /
    final_rca_frame["cost_per_unit_prev"]
)

# Contribution shares (absolute, normalized)
impact_abs_sum_qc = (
    final_rca_frame["quantity_impact"].abs() +
    final_rca_frame["cost_impact"].abs()
)

impact_abs_sum_all = (
    impact_abs_sum_qc +
    final_rca_frame["interaction_impact"].abs()
)


# quantity_impact_pct_of_total 
final_rca_frame["quantity_impact_pct_of_total"] = np.where(
    impact_abs_sum_qc > 0,
    final_rca_frame["quantity_impact"].abs() / impact_abs_sum_qc,
    0
)

# cost_impact_pct_of_total 
final_rca_frame["cost_impact_pct_of_total"] = np.where(
    impact_abs_sum_qc > 0,
    final_rca_frame["cost_impact"].abs() / impact_abs_sum_qc,
    0
)

# interaction_pct
final_rca_frame["interaction_pct"] = np.where(
    impact_abs_sum_all > 0,
    final_rca_frame["interaction_impact"].abs() / impact_abs_sum_all,
    0
)


# Dominance Score

final_rca_frame["abs_qty_impact"] = final_rca_frame["quantity_impact"].abs()
final_rca_frame["abs_cost_impact"] = final_rca_frame["cost_impact"].abs()

final_rca_frame["dominance_score"] = np.where(
    (final_rca_frame["abs_qty_impact"] + final_rca_frame["abs_cost_impact"]) == 0,
    0.0,  # avoid divide-by-zero → treat as neutral
    (final_rca_frame["abs_qty_impact"] - final_rca_frame["abs_cost_impact"]) /
    (final_rca_frame["abs_qty_impact"] + final_rca_frame["abs_cost_impact"])
)

In [295]:
final_rca_frame.head()

,corporate_brand,material_type,development_lifecycle_status,enterprise_category,enterprise_sub_category,dosage_form_parent,corp_brand_id,network_or_business_unit,profit_center,brand_name,...,interaction_impact,total_fip_change,delta_quantity_pct,delta_cost_per_unit_pct,quantity_impact_pct_of_total,cost_impact_pct_of_total,interaction_pct,abs_qty_impact,abs_cost_impact,dominance_score
0,REVLIMID,PACK,COMMERCIAL,PACKAGE COMPONENT,PACKAGE COMPONENT,nan,03302101,PHARMA,10956,REVLIMID,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
1,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
2,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
3,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10001,REVLIMID,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
4,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10001,REVLIMID,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN


In [296]:
final_rca_frame.columns

Index(['corporate_brand', 'material_type', 'development_lifecycle_status',
       'enterprise_category', 'enterprise_sub_category', 'dosage_form_parent',
       'corp_brand_id', 'network_or_business_unit', 'profit_center',
       'brand_name', 'material_description', 'material_group',
       'old_material_number', 'base_unit_of_measure', 'unit_of_weight',
       'plant_specific_material_status', 'mrp_type', 'procurement_type',
       'safety_stock', 'minimum_lot_size', 'maximum_lot_size',
       'fixed_lot_size', 'total_replenishment_lead_time', 'total_shelf_life',
       'batch_management', 'abc_indicator', 'source', 'concost_source',
       'Plant Type', 'nodetype', 'unit_of_measure', 'material', 'plant',
       'date', 'quantity_curr', 'total_cost_curr', 'cost_per_unit_curr',
       'snapshot_type', 'snapshot_date', 'sku_status', 'prev_snapshot_date',
       'snapshot_date_prev', 'quantity_prev', 'cost_per_unit_prev',
       'total_cost_prev', 'is_new_in_current_snapshot',
       'i

In [297]:
final_rca_frame = final_rca_frame.sort_values(
    ["material", "plant", "snapshot_date"]
)

final_rca_frame["hist_count"] = (
    final_rca_frame
    .groupby(["material", "plant"])
    .cumcount()
)

final_rca_frame["has_sufficient_6periods"] = (
    final_rca_frame["hist_count"] >= 6 )


In [298]:
# Step 2 - Noise vs Signal determination 


# Sorting

final_rca_frame = final_rca_frame.sort_values(
    ["material", "plant", "snapshot_date"]
)

#  1) Business materiality (value-based)   
# How big the change is in value terms, relative to prior fip.
final_rca_frame["relative_fip_impact"] = np.where(
    final_rca_frame["total_cost_prev"] > 0,
    final_rca_frame["total_fip_change"].abs() /
    final_rca_frame["total_cost_prev"],
    np.nan
)


# # 2) Historical volatility (material–plant aware)

# # Quantity volatility (rolling, 12 snapshots) - How noisy this SKU normally is.
# # quantity_volatility_12w = std(delta_quantity_pct)
# final_rca_frame["quantity_volatility_12w"] = (
#     final_rca_frame
#     .groupby(["material", "plant"])["delta_quantity_pct"]
#     .transform(lambda x: x.shift(1).rolling(12, min_periods=6).std()) 
# )


# # Cost volatility (rolling, 12 snapshots)
# final_rca_frame["cost_volatility_12w"] = (
#     final_rca_frame
#     .groupby(["material", "plant"])["delta_cost_per_unit_pct"]
#     .transform(lambda x: x.shift(1).rolling(12, min_periods=6).std())
# )


# # 3) Normalized (volatility-aware) changes

# final_rca_frame["normalized_quantity_change"] = (
#     final_rca_frame["delta_quantity_pct"] /
#     final_rca_frame["quantity_volatility_12w"]
# )

# final_rca_frame["normalized_cost_change"] = (
#     final_rca_frame["delta_cost_per_unit_pct"] /
#     final_rca_frame["cost_volatility_12w"]
# )

# # Interpretation:
# # ≈ 1 → normal
# # ≫ 1 → unusually large vs history


In [299]:
# Temporal persistence (directional consistency)

final_rca_frame["quantity_change_sign"] = np.sign(
    final_rca_frame["delta_quantity_pct"]
) 

def persistence_score(series):
    score = []
    current = 0
    prev = 0
    for v in series:
        if v == 0 or pd.isna(v):
            current = 0
        elif v == prev:
            current += 1
        else:
            current = 1
        score.append(current)
        prev = v
    return score

final_rca_frame["quantity_persistence_score"] = (
    final_rca_frame
    .groupby(["material", "plant"])["quantity_change_sign"]
    .transform(persistence_score)
)


In [300]:
# Oultier Check

ROLLING_WINDOW = 12
MIN_PERIODS = 6
EXTREME_PCT_CHANGE = 0.5     # 50% absolute change
LOW_LEVEL_RATIO = 0.01       
ROBUST_Z_THRESHOLD = 3

# 1) Rolling median of quantity (baseline scale)

final_rca_frame["quantity_rolling_median_12w"] = (
    final_rca_frame
    .groupby(["material", "plant"])["quantity_curr"]
    .transform(lambda x: x.shift(1).rolling(ROLLING_WINDOW, min_periods=MIN_PERIODS).median())
)

# 2) Structural extreme change (scale-based)
final_rca_frame["is_structural_extreme_qty"] = (
    (final_rca_frame["delta_quantity_pct"].abs() >= EXTREME_PCT_CHANGE) &
    (
        (final_rca_frame["quantity_curr"] <=
         LOW_LEVEL_RATIO * final_rca_frame["quantity_rolling_median_12w"]) |     
        (final_rca_frame["quantity_curr"] >=
         (1 / LOW_LEVEL_RATIO) * final_rca_frame["quantity_rolling_median_12w"])
    )
)

# 3) Create CLEAN delta series (exclude known outliers)
final_rca_frame["clean_delta_quantity_pct"] = final_rca_frame["delta_quantity_pct"]

final_rca_frame.loc[
    final_rca_frame["is_structural_extreme_qty"],
    "clean_delta_quantity_pct"
] = np.nan

# 4) Robust MAD-based z-score (on clean history only)
def robust_zscore(series):
    median = np.nanmedian(series)
    mad = np.nanmedian(np.abs(series - median))
    if mad == 0 or np.isnan(mad):
        return pd.Series(np.nan, index=series.index)
    return (series - median) / (1.4826 * mad)

final_rca_frame["robust_quantity_z"] = (
    final_rca_frame
    .groupby(["material", "plant"])["clean_delta_quantity_pct"]
    .transform(lambda x: robust_zscore(x.shift(1)))
)

# 5) Statistical outlier (robust)
final_rca_frame["is_statistical_outlier_qty"] = (
    final_rca_frame["robust_quantity_z"].abs() > ROBUST_Z_THRESHOLD
)

# 6) FINAL quantity outlier flag
final_rca_frame["is_quantity_outlier"] = (
    final_rca_frame["is_structural_extreme_qty"] |
    final_rca_frame["is_statistical_outlier_qty"]
)

# 7) Cleanup (optional but recommended)
bool_cols = [
    "is_structural_extreme_qty",
    "is_statistical_outlier_qty",
    "is_quantity_outlier",
]

final_rca_frame[bool_cols] = (
    final_rca_frame[bool_cols]
    .fillna(False)
    .astype("boolean")
)


/tmp/ipykernel_11579/3852578463.py:38: RuntimeWarning: All-NaN slice encountered
  median = np.nanmedian(series)
/tmp/ipykernel_11579/3852578463.py:39: RuntimeWarning: All-NaN slice encountered
  mad = np.nanmedian(np.abs(series - median))


In [152]:
# # Outlier new

# # rolling median
# final_rca_frame["quantity_rolling_median_12w"] = (
#     final_rca_frame
#     .groupby(["material", "plant"])["quantity_curr"]
#     .transform(lambda x: x.shift(1).rolling(12, min_periods=6).median())
# )

# EXTREME_PCT_CHANGE = 0.5        # 50% change
# LOW_LEVEL_RATIO = 0.01          # 1% of typical level
# Z_SCORE_THRESHOLD = 3

# final_rca_frame["is_structural_extreme_qty"] = (
#     (final_rca_frame["delta_quantity_pct"].abs() >= EXTREME_PCT_CHANGE) &
#     (
#         final_rca_frame["quantity_curr"] <=
#         LOW_LEVEL_RATIO * final_rca_frame["quantity_rolling_median_12w"]
#     )
# )

# final_rca_frame["is_statistical_outlier_qty"] = (
#     (final_rca_frame["quantity_volatility_12w"] > 0) &
#     (final_rca_frame["normalized_quantity_change"].abs() > Z_SCORE_THRESHOLD)
# )

# final_rca_frame["is_quantity_outlier"] = (
#     final_rca_frame["is_structural_extreme_qty"] |
#     final_rca_frame["is_statistical_outlier_qty"]
# )


In [153]:
# # 5) Outlier detection (z-score based - per material–plant) -old calc

# final_rca_frame["quantity_zscore"] = (
#     final_rca_frame["delta_quantity_pct"] /       # check
#     final_rca_frame["quantity_volatility_12w"]
# )

# final_rca_frame["is_quantity_outlier"] = (
#     final_rca_frame["quantity_zscore"].abs() > 3  # threshold
# )


In [154]:
# # 6) Change-point detection

# final_rca_frame["rolling_avg_quantity_12w"] = (
#     final_rca_frame
#     .groupby(["material", "plant"])["delta_quantity_pct"]
#     .transform(lambda x: x.shift(1).rolling(12, min_periods=3).mean())
# )

# final_rca_frame["change_point_detected"] = (
#     (final_rca_frame["quantity_persistence_score"] >= 2) &
#     (final_rca_frame["rolling_avg_quantity_12w"].abs() > 
#      final_rca_frame["quantity_volatility_12w"])
# )

In [301]:
# Change Point

final_rca_frame["rolling_avg_quantity_12w"] = (
    final_rca_frame
    .groupby(["material", "plant"])["delta_quantity_pct"]
    .transform(lambda x: x.shift(1).rolling(12, min_periods=6).mean())
)


# Robust rolling MAD of delta quantity
final_rca_frame["delta_qty_rolling_mad_12w"] = (
    final_rca_frame
    .groupby(["material", "plant"])["delta_quantity_pct"]
    .transform(
        lambda x: x.shift(1)
                  .rolling(12, min_periods=6)
                  .apply(lambda s: np.nanmedian(np.abs(s - np.nanmedian(s))), raw=True)
    )
)


# Robust trend strength
final_rca_frame["robust_trend_strength"] = (
    final_rca_frame["rolling_avg_quantity_12w"].abs() /
    (1.4826 * final_rca_frame["delta_qty_rolling_mad_12w"])
)


# Short-window Trend confirmation
final_rca_frame["rolling_avg_quantity_6w"] = (
    final_rca_frame
    .groupby(["material", "plant"])["delta_quantity_pct"]
    .transform(lambda x: x.shift(1).rolling(6, min_periods=3).mean())
)

final_rca_frame["trend_confirmed"] = (
    np.sign(final_rca_frame["rolling_avg_quantity_12w"]) ==
    np.sign(final_rca_frame["rolling_avg_quantity_6w"])
)


# FINAL robust change-point detection
final_rca_frame["change_point_detected"] = (
    (final_rca_frame["quantity_persistence_score"] >= 2) &
    (final_rca_frame["robust_trend_strength"] > 2) &
    (final_rca_frame["trend_confirmed"])
)


In [302]:
# Final Noise Vs Signal Flag

MATERIALITY_THRESHOLD = 0.5
MIN_PERSISTENCE = 2

final_rca_frame["is_noise"] = (
    final_rca_frame["is_quantity_outlier"] &
    (final_rca_frame["quantity_persistence_score"] < MIN_PERSISTENCE) &
    (~final_rca_frame["change_point_detected"])
)

final_rca_frame["is_large_noise"] = (
    final_rca_frame["is_noise"] &
    (final_rca_frame["relative_fip_impact"] > 0.5)
)

final_rca_frame.loc[
    ~final_rca_frame["has_sufficient_6periods"],
    "is_noise"
] = False                                             # insufficient data to signal

final_rca_frame["is_signal"] = ~final_rca_frame["is_noise"]


In [304]:
final_rca_frame["abs_fip_change"] = final_rca_frame["total_fip_change"].abs()

final_rca_frame["snapshot_signal_rank"] = (
    final_rca_frame
    .where(final_rca_frame["is_signal"])
    .groupby(["snapshot_date", "corporate_brand"])["abs_fip_change"]
    .rank(method="first", ascending=False)
)
final_rca_frame["is_top10_contributor"] = (
    final_rca_frame["snapshot_signal_rank"] <= 10
)

final_rca_frame = final_rca_frame.drop(["snapshot_signal_rank","abs_fip_change"],axis=1)

In [305]:
final_rca_frame.head()

,corporate_brand,material_type,development_lifecycle_status,enterprise_category,enterprise_sub_category,dosage_form_parent,corp_brand_id,network_or_business_unit,profit_center,brand_name,...,rolling_avg_quantity_12w,delta_qty_rolling_mad_12w,robust_trend_strength,rolling_avg_quantity_6w,trend_confirmed,change_point_detected,is_noise,is_large_noise,is_signal,is_top10_contributor
811,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,False,False,False,False,True,False
1106,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,False,False,False,False,True,False
2079,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,False,False,False,False,True,False
3098,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,False,False,False,False,True,False
4264,REVLIMID,FIN,COMMERCIAL,MARKET UNIT,SINGLE PRODUCT,CAPSULE,03302101,PHARMA,10013,REVLIMID,...,NaN,NaN,NaN,NaN,False,False,False,False,True,False


In [306]:
final_rca_frame.columns

Index(['corporate_brand', 'material_type', 'development_lifecycle_status',
       'enterprise_category', 'enterprise_sub_category', 'dosage_form_parent',
       'corp_brand_id', 'network_or_business_unit', 'profit_center',
       'brand_name', 'material_description', 'material_group',
       'old_material_number', 'base_unit_of_measure', 'unit_of_weight',
       'plant_specific_material_status', 'mrp_type', 'procurement_type',
       'safety_stock', 'minimum_lot_size', 'maximum_lot_size',
       'fixed_lot_size', 'total_replenishment_lead_time', 'total_shelf_life',
       'batch_management', 'abc_indicator', 'source', 'concost_source',
       'Plant Type', 'nodetype', 'unit_of_measure', 'material', 'plant',
       'date', 'quantity_curr', 'total_cost_curr', 'cost_per_unit_curr',
       'snapshot_type', 'snapshot_date', 'sku_status', 'prev_snapshot_date',
       'snapshot_date_prev', 'quantity_prev', 'cost_per_unit_prev',
       'total_cost_prev', 'is_new_in_current_snapshot',
       'i

In [307]:
v = final_rca_frame[(final_rca_frame["corporate_brand"]=='REVLIMID') & (final_rca_frame ["snapshot_date"] > "2025-11-09") 
    &   (final_rca_frame ["material"] == '1456877')]
display(v)

,corporate_brand,material_type,development_lifecycle_status,enterprise_category,enterprise_sub_category,dosage_form_parent,corp_brand_id,network_or_business_unit,profit_center,brand_name,...,rolling_avg_quantity_12w,delta_qty_rolling_mad_12w,robust_trend_strength,rolling_avg_quantity_6w,trend_confirmed,change_point_detected,is_noise,is_large_noise,is_signal,is_top10_contributor
16594,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,-0.333333,0.000000,inf,NaN,False,False,True,True,False,False
17585,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,9.136148,0.500000,1.232449e+01,18.938963,True,True,False,False,True,True
18554,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,NaN,NaN,NaN,18.938963,False,False,False,False,True,True
19183,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,NaN,NaN,NaN,14.377967,False,False,False,False,True,True
19926,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,NaN,NaN,NaN,11.502093,False,False,False,False,True,True
21401,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,9.418443,0.348192,1.824468e+01,11.302131,True,False,True,True,False,False
21694,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,8079.358665,0.694980,7.841163e+03,11311.102131,True,True,False,False,True,True
22802,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,7315.213554,0.848096,5.817789e+03,11692.978308,True,False,False,False,True,False
23947,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,6502.412048,0.694980,6.310708e+03,9744.148590,True,False,False,False,True,False
24056,REVLIMID,HALB,COMMERCIAL,API,API / DRUG SUBSTANCE,nan,03302101,PHARMA,10959,REVLIMID,...,5852.170843,0.348192,1.133638e+04,9744.032760,True,False,False,False,True,True


In [308]:
cols = ['material', 'plant', 'date', 'quantity_curr','quantity_prev', 'total_cost_curr','cost_per_unit_curr',
    "snapshot_date",'quantity_persistence_score','change_point_detected', 'is_noise', 'is_large_noise', 'is_signal','is_top10_contributor']

v1 = final_rca_frame.loc[
    (final_rca_frame["snapshot_date"] > "2025-11-09") &
    (final_rca_frame["material"] == "1456877"),
    cols
]
v1

,material,plant,date,quantity_curr,quantity_prev,total_cost_curr,cost_per_unit_curr,snapshot_date,quantity_persistence_score,change_point_detected,is_noise,is_large_noise,is_signal,is_top10_contributor
16594,1456877,2091,202612,3.882051e+06,6.714390e+04,3.677588e+07,9.47331,2025-11-14,1,False,True,True,False,False
17585,1456877,2091,202612,3.618853e+04,0.000000e+00,3.428252e+05,9.47331,2025-11-19,2,True,False,False,True,True
18554,1456877,2091,202612,6.580000e+06,3.882051e+06,6.233438e+07,9.47331,2025-11-21,3,False,False,False,True,True
19183,1456877,2091,202612,6.570763e+06,6.580000e+06,6.224687e+07,9.47331,2025-11-28,1,False,False,False,True,True
19926,1456877,2091,202612,1.260000e+03,6.570763e+06,1.193637e+04,9.47331,2025-12-05,2,False,False,False,True,True
21401,1456877,2091,202612,7.119000e+07,1.260000e+03,6.744049e+08,9.47331,2025-12-12,1,False,True,True,False,False
21694,1456877,2091,202612,7.119000e+07,3.618853e+04,6.744049e+08,9.47331,2025-12-17,2,True,False,False,True,True
22802,1456877,2091,202612,7.119000e+07,7.119000e+07,6.744049e+08,9.47331,2025-12-19,0,False,False,False,True,False
23947,1456877,2091,202612,7.119000e+07,7.119000e+07,6.744049e+08,9.47331,2025-12-26,0,False,False,False,True,False
24056,1456877,2091,202612,7.119000e+07,7.119000e+07,1.482382e+09,20.82289,2026-01-02,0,False,False,False,True,True


In [89]:
# 1478550 @1037

In [54]:
final_rca_frame.to_csv('revlimid_output.csv')

In [ ]:
# cols_to_remove = [
#     'development_lifecycle_status',
#        'enterprise_category', 'enterprise_sub_category', 'dosage_form_parent',
#        'corp_brand_id', 'network_or_business_unit'
# ]

# df_final = final_rca_frame.drop(columns=cols_to_remove, errors="ignore")



# v = final_rca_frame[(final_rca_frame["corporate_brand"]=='REVLIMID') & (final_rca_frame ["snapshot_date"] == "2026-01-09") 
#     &   (final_rca_frame ["material"] == '1456877')]